In [1]:
import os 
# 这三行必须放在最上面，加载任何 huggingface 东西之前！
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"  # 关键！国内加速镜站
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "300"  # 超时改成 5 分钟
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.expanduser("~/huggingface_cache")

from langchain_classic.retrievers import EnsembleRetriever, BM25Retriever
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_cohere import CohereRerank
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

# Vector Retriever

In [2]:
presist_directory = "db/chroma_db"
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)

db = Chroma(
    persist_directory=presist_directory,
    embedding_function=embedding_model,
    collection_metadata={"hnsw:space": "cosine"},
)

vector_retriever = db.as_retriever(search_kwargs={"k": 10})

# BM25 Retriever

In [3]:
documents = db.get()["documents"]
bm25_retriever = BM25Retriever.from_texts(documents)
bm25_retriever.k = 10

# Hybrid Retriever

In [4]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.7, 0.3]
)

In [9]:
query = "龙二刚开始是怎么和富贵认识的？龙二最后的结局是怎样的？"
hybrid_result = hybrid_retriever.invoke(query)
print(type(hybrid_result[0]))
for i, doc in enumerate(hybrid_result):
    print("="*20, f"Document-{i+1}", "="*20)
    print(doc)

<class 'langchain_core.documents.base.Document'>
==================== Document-1 ====================
page_content='"如今屋子和地都是龙二的了，家安在这里跟安在别处也一样。"
　　我娘听了这话，过了半晌才说：
　　"你爹的坟还在这里。"
　　我娘一句话就让我不敢再想别的主意了，我想来想去只好去找龙二。
　　龙二成了这里的地主，常常穿着丝绸衣衫，右手拿着茶壶在田埂上走来走去，神气得很。镶着两颗大金牙的嘴总是咧开笑着，有时骂看着不顺眼的佃户时也咧着嘴，我起先还以为他对人亲热，慢慢地就知道他是要别人都看到他的金牙。
　　龙二遇到我还算客气，常笑嘻嘻地说：
　　"福贵，到我家来喝壶茶吧。"
　　我一直没去龙二家是怕自己心里发酸，我两脚一落地就住在那幢屋子里了，如今那屋子是龙二的家，你想想我心里是什么滋味。
　　其实人落到那种地步也就顾不上那么多了，我算是应了人穷志短那句古话了。那天我去找龙二时，龙二坐在我家客厅的太师椅子里，两条腿搁在凳子上，一手拿茶壶一手拿着扇子，看到我走进来，龙二咧嘴笑道：
　　"是福贵，自己找把凳子坐吧。"
　　他躺在太师椅里动都没动，我也就不指望他泡壶茶给我喝。我坐下后龙二说：
　　"福贵，你是来找我借钱的吧？"
　　我还没说不是，他就往下说道：
　　"按理说我也该借几个钱给你，俗话说是救急不救穷，我啊，只能救你的急，不会救你的穷。"
　　我点点头说："我想租几亩田。"
　　龙二听后笑眯眯地问：
　　"你要租几亩？"
　　我说："租五亩。"
　　"五亩？"龙二眉毛往上吊了吊，问："你这身体能行吗？"
　　我说："练练就行了。"
　　他想一想说："我们是老相识了，我给你五亩好田。"' metadata={'source': 'docs/余华 活着.txt'}
==================== Document-2 ====================
page_content='"福贵，我是替你去死啊。"
　　听他这么一喊，我慌了，想想还是离开吧，别看他怎么死了。我从人堆里挤出去，一个人往外走，走了十来步就听到"电"的一枪，我想龙二彻底完蛋了，可紧接着又是"电"的一枪，下面又打了三枪，总共是五枪。我想是不是还有别的人也给毙掉，

In [11]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=reranker_model, top_n=5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=hybrid_retriever
)
docs = compression_retriever.invoke(query)

# compressor = CohereRerank(model="rerank-v3.5", top_n=6)
# docs = compressor.compress_documents(hybrid_result, query)

for i, doc in enumerate(docs):
    print("="*20, f"Document-{i+1}", "="*20)
    print(doc)

==================== Document-1 ====================
page_content='"如今屋子和地都是龙二的了，家安在这里跟安在别处也一样。"
　　我娘听了这话，过了半晌才说：
　　"你爹的坟还在这里。"
　　我娘一句话就让我不敢再想别的主意了，我想来想去只好去找龙二。
　　龙二成了这里的地主，常常穿着丝绸衣衫，右手拿着茶壶在田埂上走来走去，神气得很。镶着两颗大金牙的嘴总是咧开笑着，有时骂看着不顺眼的佃户时也咧着嘴，我起先还以为他对人亲热，慢慢地就知道他是要别人都看到他的金牙。
　　龙二遇到我还算客气，常笑嘻嘻地说：
　　"福贵，到我家来喝壶茶吧。"
　　我一直没去龙二家是怕自己心里发酸，我两脚一落地就住在那幢屋子里了，如今那屋子是龙二的家，你想想我心里是什么滋味。
　　其实人落到那种地步也就顾不上那么多了，我算是应了人穷志短那句古话了。那天我去找龙二时，龙二坐在我家客厅的太师椅子里，两条腿搁在凳子上，一手拿茶壶一手拿着扇子，看到我走进来，龙二咧嘴笑道：
　　"是福贵，自己找把凳子坐吧。"
　　他躺在太师椅里动都没动，我也就不指望他泡壶茶给我喝。我坐下后龙二说：
　　"福贵，你是来找我借钱的吧？"
　　我还没说不是，他就往下说道：
　　"按理说我也该借几个钱给你，俗话说是救急不救穷，我啊，只能救你的急，不会救你的穷。"
　　我点点头说："我想租几亩田。"
　　龙二听后笑眯眯地问：
　　"你要租几亩？"
　　我说："租五亩。"
　　"五亩？"龙二眉毛往上吊了吊，问："你这身体能行吗？"
　　我说："练练就行了。"
　　他想一想说："我们是老相识了，我给你五亩好田。"' metadata={'source': 'docs/余华 活着.txt'}
==================== Document-2 ====================
page_content='我离家两个月多一点，我娘就死了。家珍告诉我，我娘死前一遍一遍对家珍说：
　　"福贵不会是去赌钱的。"
　　家珍去城里打听过我不知多少次，竟会没人告诉她我被抓了壮丁。我娘才这么说，可怜她死的时候，还不知道我在什么地方。我的凤霞也可怜，一年前她发了一次高烧后就再不会说话了。家珍哭着告诉我这些时，凤霞就坐在我对面，她知道我们是在说她，就轻轻地对着我笑，看

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage
model = ChatOpenAI(model="doubao-seed-1-6-lite-251015")
combined_input = f'''请结合文档中的内容，回答我的问题。
文档内容如下：
{chr(10).join([f"{doc.page_content}" for doc in docs])}
我的问题是：{query}
'''

messages = [
    SystemMessage(content="你是一个乐于助人的助手！"),
    HumanMessage(content=combined_input),
]
results = model.invoke(messages)
print(results.content)

### 龙二与福贵的初识  
龙二最初因**福贵赌钱**结识：福贵作为徐家少爷时沉迷赌博，将家中房产、地产抵押输掉，龙二作为赌局赢家买下了这些产业，由此与福贵产生交集。


### 龙二的结局  
土地改革时期，龙二因被认定为**恶霸地主**，被人民政府逮捕；最终在邻村被枪决，挨了五枪。


（依据文档内容：福贵赌输家产后龙二购得产业；土地改革时龙二“死不认帐”“吓唬佃户”被抓，最终被枪决，共挨五枪。）
